# Machine Learning - Project 1

## Imports

In [7]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from implementations import *
from helpers import *
from proj1_helpers import *
%load_ext autoreload
%autoreload 2

In [8]:
y, tx, ids = load_csv_data('../data/train.csv', sub_sample=False)

## Data Analysis

We verified that the data is divided by categories indicated by column PRI_jet_num, which will be helpful to perform the preprocessing and data cleaning to deal the NaN values in the data.

### Category analysis:

We noticed that there is a feature `PRI_jet_num` that categorizes the data. The Nan values vary according to its value. There are mainly 4 possibilities:

In [9]:
#22 is the column that indicates the category
cat_index = 22

cat0_row_indices = np.where(tx[:,cat_index] == 0)[0] 
cat1_row_indices = np.where(tx[:,cat_index] == 1)[0] 
cat2_row_indices = np.where(tx[:,cat_index] == 2)[0] 
cat3_row_indices = np.where(tx[:,cat_index] == 3)[0] 

Let's analyse the columns with NaN values for each category

In [10]:
cat0_NaNcol = np.unique(np.where(tx[np.where(tx[:, cat_index] == 0)[0], :] == -999)[1])
cat1_NaNcol = np.unique(np.where(tx[np.where(tx[:, cat_index] == 1)[0], :] == -999)[1])
cat2_NaNcol = np.unique(np.where(tx[np.where(tx[:, cat_index] == 2)[0], :] == -999)[1])
cat3_NaNcol = np.unique(np.where(tx[np.where(tx[:, cat_index] == 3)[0], :] == -999)[1])
    
#we see the columns for which there are NaN for each category.
print(" cat0_NaNcol:", cat0_NaNcol,"\n", "cat1_NaNcol:", cat1_NaNcol, "\n", "cat2_NaNcol:", cat2_NaNcol, 
      "\n", "cat3_NaNcol:", cat3_NaNcol, "\n")


 cat0_NaNcol: [ 0  4  5  6 12 23 24 25 26 27 28] 
 cat1_NaNcol: [ 0  4  5  6 12 26 27 28] 
 cat2_NaNcol: [0] 
 cat3_NaNcol: [0] 



In [11]:
set(cat0_NaNcol) - set(cat1_NaNcol)

{23, 24, 25}

In [12]:
len(cat0_NaNcol)

11

Now, we need to check the `percentage of NaN values` of each column per category, in order to decide how to pre-process the data

- Category 0:

In [13]:
percentage_NaN = np.asarray([len(np.where(tx[cat0_row_indices, i] == -999)[0]) \
                             /len(cat0_row_indices) for i in cat0_NaNcol]) * 100
list_ = list(zip(cat0_NaNcol, percentage_NaN))  #(column i (has -999), percentage of -999 in that column)

print("List output format: (column i has NaN value, percentage of NaN in column i)")
[print(i) for i in list_]
print("\n")

List output format: (column i has NaN value, percentage of NaN in column i)
(0, 26.145746799715752)
(4, 100.0)
(5, 100.0)
(6, 100.0)
(12, 100.0)
(23, 100.0)
(24, 100.0)
(25, 100.0)
(26, 100.0)
(27, 100.0)
(28, 100.0)




- Category 1:

In [14]:
percentage_NaN = np.asarray([len(np.where(tx[cat1_row_indices , i] == -999)[0]) \
                             /len(cat1_row_indices) for i in cat1_NaNcol]) * 100
list_ = list(zip(cat1_NaNcol, percentage_NaN))  #(column i (has -999), percentage of -999 in that column)

print("List output format: (column i has NaN value, percentage of NaN in column i)")
[print(i) for i in list_]
print("\n")

List output format: (column i has NaN value, percentage of NaN in column i)
(0, 9.7518828020220774)
(4, 100.0)
(5, 100.0)
(6, 100.0)
(12, 100.0)
(26, 100.0)
(27, 100.0)
(28, 100.0)




- Category 2:

In [15]:
percentage_NaN = np.asarray([len(np.where(tx[cat2_row_indices , i] == -999)[0]) \
                             /len(cat2_row_indices) for i in cat2_NaNcol]) * 100
list_ = list(zip(cat2_NaNcol, percentage_NaN))  #(column i (has -999), percentage of -999 in that column)

print("List output format: (column i has NaN value, percentage of NaN in column i)")
[print(i) for i in list_]
print("\n")

List output format: (column i has NaN value, percentage of NaN in column i)
(0, 5.8595843506222831)




- Category 3:

In [16]:
percentage_NaN = np.asarray([len(np.where(tx[cat3_row_indices , i] == -999)[0]) \
                             /len(cat3_row_indices) for i in cat3_NaNcol]) * 100
list_ = list(zip(cat3_NaNcol, percentage_NaN))  #(column i (has -999), percentage of -999 in that column)

print("List output format: (column i has NaN value, percentage of NaN in column i)")
[print(i) for i in list_]
print("\n")

List output format: (column i has NaN value, percentage of NaN in column i)
(0, 6.6639595740841013)




We conclude that for each category, all the columns with NaN values, are 100% filled with NaN values, except for the first column, which  has NaN values distributed by all categories. furthermore, the total percentage of NaN values for the first column is low, therefore, we decided to replace the NaN values in this column by the average of the cleaned lines.

In [17]:
cleaned_lines_index = np.where(tx[:, 0] != -999)[0]
avg = np.mean(tx[cleaned_lines_index, 0])
NaN_lines_index = np.where(tx[:, 0] == -999)[0]
tx[NaN_lines_index, 0] = avg*np.ones(len(NaN_lines_index))

Thus, after this procedure, we will also remove this column from the NaN column list for each category.

In [18]:
cat0_NaNcol = cat0_NaNcol[1:]
cat1_NaNcol = cat1_NaNcol[1:]
cat2_NaNcol = cat2_NaNcol[1:]
cat3_NaNcol = cat3_NaNcol[1:]

In [19]:
with open('catNaNcol.pickle', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([cat0_NaNcol, cat1_NaNcol, cat2_NaNcol, cat3_NaNcol], f)

### Set Creation by category

In this section we separate the data into categories and clean -999 columns except first column (only 15% NaN values)

In [39]:
tx_cat0 = tx[cat0_row_indices, :]
y_cat0  = y[cat0_row_indices]

tx_cat1 = tx[cat1_row_indices, :]
y_cat1  = y[cat1_row_indices]

tx_cat2 = tx[cat2_row_indices, :]
y_cat2  = y[cat2_row_indices]

tx_cat3 = tx[cat3_row_indices, :]
y_cat3  = y[cat3_row_indices]

We also concluded from the previous section that category 2 and 3 are basically the same. From now one category 2 = category 2 + category 3 (by merging them)

In [40]:
tx_cat2 = np.vstack((tx_cat2, tx_cat3))
y_cat2 = np.hstack((y_cat2, y_cat3))

Now, we will also add the category column to the NaNcol set so that it is deleted next (not relevant)
We verify that for category 0 the last column is also 0 always, so we eliminate it

In [41]:
cat0_toDelete = np.hstack((cat0_NaNcol, cat_index, np.array([29])))
cat1_toDelete = np.hstack((cat1_NaNcol, cat_index))
cat2_toDelete = np.array([cat_index])

## Data cleaning

Delete NaN columns for each tx set:

In [42]:
tx_cat0 = np.delete(tx_cat0, cat0_toDelete, 1)
tx_cat1 = np.delete(tx_cat1, cat1_toDelete, 1)
tx_cat2 = np.delete(tx_cat2, cat2_toDelete, 1)

y_cat = [y_cat0, y_cat1, y_cat2]
tx_cat = [tx_cat0, tx_cat1, tx_cat2]

### Mutual Information between output and features

In this section, we calculate mutual information between input features and output in order to evaluate importance of each feature.

In [43]:
#Calculate mutual information between y and each feature

def shan_entropy(c):
    c_normalized = c / float(np.sum(c))
    c_normalized = c_normalized[np.nonzero(c_normalized)]
    H = -sum(c_normalized* np.log2(c_normalized))  
    return H

def calc_MI(X,Y, bins, bins_Y):
    c_XY = np.histogram2d(X, Y, bins)[0]
    c_X = np.histogram(X, bins)[0]
    c_Y = np.histogram(Y, bins_Y)[0]

    H_X = shan_entropy(c_X)
    H_Y = shan_entropy(c_Y)
    H_XY = shan_entropy(c_XY)

    MI = H_X + H_Y - H_XY
    return MI

In [44]:
top_features = []
other_features = []
max_features = 9

We create a function that calculates mutual information between output and each feature for all sets. 
It returns two lists, one with most important features and the other one with the remaining ones for each set.

In [45]:
print("List output format: (column i , Mutual information with label)", "\n")
def mutual_information():
    bins = 50
    
    for i in range(len(tx_cat)):
    
        # Standardize Data, for iterative methods
        tx_std, mean_x, std_x = standardize(tx_cat[i])
        y = y_cat[i]

        vecMI = np.zeros(np.shape(tx_std)[1])

        for j in range(np.shape(tx_std)[1]):
            vecMI[j] = calc_MI(tx_std[:, j], y, bins, 2)

        top_features.append(np.argsort(-vecMI)[:max_features])
        other_features.append(np.argsort(-vecMI)[max_features:])
        
        print('Top features for category {}: {}'.format(i,top_features[i]))
        
    return top_features, other_features

List output format: (column i , Mutual information with label) 



In [46]:
top_features, other_features = mutual_information()

Top features for category 0: [ 1  7  4  9  2  0  6  8 12]
Top features for category 1: [ 0  2  1  8  9  7  4  6 19]
Top features for category 2: [ 0  2  4  6  5 12  1 11 26]


In [47]:
import pickle

# Saving the objects:
with open('top_features.pickle', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([top_features, other_features], f)

We check that the first feature has a high relevance. On the other hand, columns 4, 12, 26, 27, 28 don't have as much importance.

### Detect Outliers

Let's detect the possible outliers in out training data, and saved their row indices for posterior cleaning. 
We consider as outliers those values who vary from the mean value for more than 3 times the variance.

In [131]:
def detect_outliers(tx, mean, std):
    outlier_indices = np.array([]) 
    for feature in range(tx.shape[1]):
        row_indices = np.where(np.absolute(tx[:,feature]-mean[feature]) > 3*std[feature])[0]
        mask = np.in1d(row_indices, outlier_indices)
        outlier_indices = np.hstack((outlier_indices, row_indices[np.where(~mask)[0]]))

        return outlier_indices.astype(int)

Delete outliers for each category:

In [132]:
for i in range(len(tx_cat)):
    
    # Standardize Data, for iterative methods
    _, mean_x, std_x = standardize(tx_cat[i])
    y = y_cat[i]   

    print("percentage of outliers for category", i, ":")
    print(len(detect_outliers(tx_cat[i], mean_x, std_x))/ tx_cat[i].shape[0] * 100)

    outlier_indices = detect_outliers(tx_cat[i], mean_x, std_x)
    tx_cat[i] = np.delete(tx_cat[i], outlier_indices, 0)
    y_cat[i] = np.delete(y_cat[i], outlier_indices, 0)


percentage of outliers for category 0 :
1.8826378949686227
percentage of outliers for category 1 :
1.9098834210254823
percentage of outliers for category 2 :
2.06498214851881


## Predicitions on Training Data

In this section, as described in the report, we constructed our method on top of Ridge regression.

### Loss Function

We will consider the parameter "1 - accuracy" as our loss function. This function calculates the ratio of the mispredicted y size and total number of labels. [see in implementations.py]

### Standardize Data

In [133]:
# Standardize Data
for i in range(len(tx_cat)):
    tx_cat[i], _, _ = standardize(tx_cat[i])

**4. Ridge Regression**

In [14]:
#create a vector of degrees
degrees = [4, 5, 5]

#create a vector of lambdas
lambdas = np.logspace(-5, 0, 15)

#define parameter for cross-validation
k_fold = 4

#define seed
seed = 20

NameError: name 'np' is not defined

**Applying ridge regression:**
- *A brief explanation:*
This function receives as parameters the data from training.csv, a vector of degreees, a vector of lambdas, a cross validation parameter - k_fold and a seed.
The aim of this routine is to iterate over several possibilities for the degree of the predictive function, as well as the lambda weight for the regularization term. 
For each pair (degree, function), the cross validation is computed, and its parameters are then averaged, and stored in two variables (loss_matrix to store losses, and w list).
Finally, we find which parameters lead to the minimium in the matrix loss, and we follow on with those (correspondent w and degree).

In [135]:
def apply_ridge_regression(y, tx, degrees, lambdas, k_fold, seed, top_features, other_features):
    
    #create varibles to store parameters
    loss_matrix = np.zeros((len(degrees), len(lambdas))) 
    w_matrix = []
    
    #build vector of indices to use in cross validation
    k_indices = build_k_indices(y, k_fold, seed)

    #iterave over degrees and lambdas to find best pair of values:
    for degree_index, degree in enumerate(degrees):    
        #create multinomial for degree:
        multinomial_i = build_multinomial(tx, degree, top_features, other_features)     
        #list of weights (with length lambda) for degree_index:
        w_lambda = []
        
        for lambda_index, lambda_ in enumerate(lambdas):        
            w_cv = []
            loss_cv = []
            
            #perform cross validation
            for k in range(k_fold):
                w_cv_k, loss_cv_k = cross_validation(y, tx, k_indices, k, lambda_, degree, multinomial_i)
                w_cv.append(w_cv_k) 
                loss_cv.append(loss_cv_k)
                
            #average parameters
            w_cv_avg = (np.asarray(w_cv).T).mean(axis=1)
            loss_te_avg = np.mean(loss_cv)
            
            #store parameters
            w_lambda.append(w_cv_avg)
            loss_matrix[degree_index, lambda_index] = loss_te_avg
            
        w_matrix.append(w_lambda)
        print("degree: ", degree)
    
    #find minimium 
    min_index = np.argwhere(loss_matrix == np.min(loss_matrix))
    print("Ridge Regression loss* =",np.min(loss_matrix))
    print(degrees[min_index[0,0]], lambdas[min_index[0,1]])
    
    return w_matrix[min_index[0,0]][min_index[0,1]], degrees[min_index[0,0]] 

In [137]:
w_rg = []
degree_rg = []

for i in range(len(tx_cat)):
    
    w_rg_i, degree_rg_i = apply_ridge_regression(y_cat[i], tx_cat[i], [degrees[i]], lambdas, k_fold, seed,
                                                 top_features[i], other_features[i])
    w_rg.append(w_rg_i)
    degree_rg.append(degree_rg_i)

degree:  4
Ridge Regression loss* = 0.153052064632
4 4.21696503429e-05
degree:  5
Ridge Regression loss* = 0.189389955298
5 0.00177827941004
degree:  5
Ridge Regression loss* = 0.166263160858
5 0.000316227766017


In [95]:
import pickle

# Saving the objects:
with open('objs.pickle', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump([w_rg, degree_rg], f)

## RUN

**Import Data from test.csv**

In [1]:
import pickle

In [49]:
# Getting back the objects:
with open('objs.pickle', 'rb') as f:  # Python 3: open(..., 'rb')
    w_rg, degree_rg = pickle.load(f)

In [2]:
with open('top_features.pickle', 'rb') as f:  # Python 3: open(..., 'rb')
    top_features, other_features = pickle.load(f)

In [ ]:
with open('catNaNcol.pickle', 'rb') as f:  # Python 3: open(..., 'rb')
    cat0_NaNcol, cat1_NaNcol, cat2_NaNcol, cat3_NaNcol = pickle.load(f)

In [98]:
_, tx_predict, ids_predict = load_csv_data('../data/test.csv', sub_sample=False)
print(tx_predict.shape[0])

568238


### Data Analysis

According to what was done previoulsy, we analyse test data and check it has the same format

### - Category analysis:

From test data, define row indices for each category:

In [99]:
#22 is the column that indicates the category
cat_index = 22

cat0_row_indices = np.where(tx_predict[:,cat_index] == 0)[0] 
cat1_row_indices = np.where(tx_predict[:,cat_index] == 1)[0] 
cat2_row_indices = np.where(tx_predict[:,cat_index] == 2)[0] 
cat3_row_indices = np.where(tx_predict[:,cat_index] == 3)[0] 

We verified, on background, that the type of NaN values distribution is fairly the same as for the training data.
So, similarly, we "clean" the NaN values of the first column using the mean of the cleaned lines 

In [100]:
#get cleaned lines and average them
cleaned_lines_index = np.where(tx_predict[:, 0] != -999)[0]
avg = np.mean(tx_predict[cleaned_lines_index, 0])

#replace with average value calculated above
NaN_lines_index = np.where(tx_predict[:, 0] == -999)[0]
tx_predict[NaN_lines_index, 0] = avg*np.ones(len(NaN_lines_index))

### Set Creation by category

In this section we separate the data into categories and clean -999 columns except first column (only 15% NaN values)

In [104]:
tx_cat0 = tx_predict[cat0_row_indices, :]
tx_cat1 = tx_predict[cat1_row_indices, :]
tx_cat2 = tx_predict[cat2_row_indices, :]
tx_cat3 = tx_predict[cat3_row_indices, :]

As done before, we merge category 2 and 3 due to their similarity.

In [105]:
tx_cat2 = tx_predict[np.where(tx_predict[:, cat_index] >= 2)[0]]

**Define columns to delete**

Now, we will also add the category column to the NaNcol set so that it is deleted next (not relevant)
We verify that for category 0 the last column is also 0 always, so we eliminate it

In [107]:
cat0_toDelete = np.hstack((cat0_NaNcol, cat_index, np.array([29])))
cat1_toDelete = np.hstack((cat1_NaNcol, cat_index))
cat2_toDelete = np.array([cat_index])

### Data cleaning

Delete NaN columns for each tx set:

In [108]:
tx_cat0 = np.delete(tx_cat0, cat0_toDelete, 1)
tx_cat1 = np.delete(tx_cat1, cat1_toDelete, 1)
tx_cat2 = np.delete(tx_cat2, cat2_toDelete, 1)

tx_cat = [tx_cat0, tx_cat1, tx_cat2]

## Prediction

In [109]:
# Standardize Data
for i in range(len(tx_cat)):
    tx_cat[i], _, _ = standardize(tx_cat[i])

### Multinomial Expansion for each Category

In [110]:
#According to ridge regression, build multinomial for degree corresponding to minimal loss
phi_predict = []
for i in range(len(tx_cat)):
    phi_predict.append(build_multinomial(tx_cat[i], degree_rg[i], top_features[i], other_features[i]))
    print(i)
print('Phi predict', np.shape(phi_predict[1]))

0
1
2
Phi predict (175338, 1357)


### Create predictions

In [111]:
y_predict = np.zeros(tx_predict.shape[0])

category_col = tx_predict[:,cat_index]
#Merge categories 2 and 3 into one single category (2)
cat3_idx = np.where(category_col == 3)
category_col[cat3_idx] = 2*np.ones(len(cat3_idx))


for i in range(len(tx_cat)):
    y_predict[np.where(category_col == i)[0]] = predict_labels(w_rg[i], phi_predict[i])

### Export CSV file

In [112]:
create_csv_submission(ids_predict, y_predict, 'submission.csv')